In [1]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans_multi as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.utils import get_model, visualize_2motions
from tqdm import tqdm
from exit.utils import get_model, visualize_2motions, generate_src_mask, init_save_folder, uniform, cosine_schedule
from einops import rearrange, repeat
import torch.nn.functional as F
from exit.utils import base_dir
from models.vqvae_multi_v2 import VQVAE_MULTI_V2

In [14]:
class Temp:
    def __init__(self, extra_args=None):
        print('mock:: opt')
        if extra_args is not None:
            for i in extra_args:
                self.__dict__[i] = extra_args[i]
args = Temp()

args.dataname = args.dataset_name = 't2m'
args.nb_code = 8192
args.code_dim = 40
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.quantizer = 'ema_reset'
args.mu = 0.99
args.clip_dim = 512


args.embed_dim_gpt = 1280
args.block_size = 51
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4

args.total_iter = 10
args.batch_size = 1
args.vq_name ='2024-08-12-23-14-09_vq5_multi'
args.resume_trans='/home/haoyum3/MMM/output/t2m/2024-08-14-12-01-17_trans_multi_1/net_last.pth'
args.resume_pth='/home/haoyum3/MMM/output/vq/2024-08-12-23-14-09_vq5_multi/net_last.pth'
args.num_local_layer =2
args.pkeep = 0.5


mock:: opt


In [3]:
from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

  0%|          | 0/1460 [00:00<?, ?it/s]

100%|██████████| 1460/1460 [00:02<00:00, 628.02it/s]


Pointer Pointing at 0


In [4]:
# [TODO] make the 'output/' folder as arg
args.vq_dir = f'/home/haoyum3/MMM/output/vq/{args.vq_name}' #os.path.join("./dataset/KIT-ML" if args.dataname == 'kit' else "./dataset/HumanML3D", f'{args.vq_name}')
codebook_dir = f'{args.vq_dir}/codebook/'
args.resume_pth = f'{args.vq_dir}/net_last.pth'
os.makedirs(args.vq_dir, exist_ok = True)
os.makedirs(codebook_dir, exist_ok = True)

In [5]:
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, codebook_dir, unit_length=2**args.down_t, multi_sep=True)
train_loader_iter = dataset_TM_train.cycle(train_loader)

100%|██████████| 23384/23384 [00:54<00:00, 428.27it/s]


In [9]:
net = VQVAE_MULTI_V2(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,#40
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate,
                       moment={'mean': torch.from_numpy(val_loader.dataset.mean).cuda().float(), 
                        'std': torch.from_numpy(val_loader.dataset.std).cuda().float()},
                       sep_decoder=True)
                       
class VQVAE_WRAPPER(torch.nn.Module):
    def __init__(self, vqvae) :
        super().__init__()
        self.vqvae = vqvae
        
    def forward(self, *args, **kwargs):
        return self.vqvae(*args, **kwargs)
print ('loading checkpoint from {}'.format(args.resume_pth))
ckpt = torch.load(args.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['net'], strict=True)
# net = torch.nn.DataParallel(net)
net = VQVAE_WRAPPER(net)


trans_encoder = trans.Text2Motion_Transformer(vqvae=net,
                                num_vq=args.nb_code, 
                                embed_dim=args.embed_dim_gpt, 
                                clip_dim=args.clip_dim, 
                                block_size=args.block_size, 
                                num_layers=args.num_layers, 
                                num_local_layer=args.num_local_layer, 
                                n_head=args.n_head_gpt, 
                                drop_out_rate=args.drop_out_rate, 
                                fc_rate=args.ff_rate)

# [TODO] DP doesn't work with single sample of vqvae decoder in eval
# [TODO] move to [1stage] VQ stage
# net = torch.nn.DataParallel(net)
net.eval()
net.cuda()

if args.resume_trans is not None:
    print ('loading transformer checkpoint from {}'.format(args.resume_trans))
    ckpt = torch.load(args.resume_trans, map_location='cpu')
    trans_encoder.load_state_dict(ckpt['trans'], strict=True)
trans_encoder.eval()
trans_encoder.cuda()
#trans_encoder = torch.nn.DataParallel(trans_encoder)

loading checkpoint from /home/haoyum3/MMM/output/vq/2024-08-12-23-14-09_vq5_multi/net_last.pth
loading transformer checkpoint from /home/haoyum3/MMM/output/t2m/2024-08-14-12-01-17_trans_multi_1/net_last.pth


Text2Motion_Transformer(
  (trans_base): CrossCondTransBase(
    (vqvae): VQVAE_WRAPPER(
      (vqvae): VQVAE_MULTI_V2(
        (decoder_left_arm): Decoder(
          (model): Sequential(
            (0): Conv1d(8, 512, kernel_size=(3,), stride=(1,), padding=(1,))
            (1): ReLU()
            (2): Sequential(
              (0): Resnet1D(
                (model): Sequential(
                  (0): ResConv1DBlock(
                    (norm1): Identity()
                    (norm2): Identity()
                    (activation1): ReLU()
                    (activation2): ReLU()
                    (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
                    (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
                  )
                  (1): ResConv1DBlock(
                    (norm1): Identity()
                    (norm2): Identity()
                    (activation1): ReLU()
                    (activation2): ReLU()
       

In [10]:
def mask_target(idx_target,idx_pred,seq_mask_no_end,batch_size,max_len,m_tokens_len,mask_id):
    if args.pkeep == -1:
        proba = np.random.rand(1)[0]
        mask = torch.bernoulli(proba * torch.ones(idx_target[...,idx_pred].shape, device=idx_target.device))
    else:
        mask = torch.bernoulli(args.pkeep * torch.ones(idx_target[...,idx_pred].shape, device=idx_target.device))
    mask = torch.logical_or(mask, ~seq_mask_no_end).int()
    r_indices = torch.randint_like(idx_target[...,idx_pred], args.nb_code)
    input_indices = mask * idx_target[...,idx_pred] + (1 - mask) * r_indices

    rand_mask_probs = torch.zeros(batch_size, device = m_tokens_len.device).float().uniform_(0.5, 1)
    num_token_masked = (m_tokens_len * rand_mask_probs).round().clamp(min = 1)
    batch_randperm = torch.rand((batch_size, max_len), device = idx_target.device) - seq_mask_no_end.int()
    batch_randperm = batch_randperm.argsort(dim = -1)
    mask_token = batch_randperm < rearrange(num_token_masked, 'b -> b 1')

    masked_input_indices = torch.where(mask_token, mask_id, input_indices)
    idx_target[...,idx_pred] = masked_input_indices
    return mask_token

def mask_else(idx_target,idx_pred,seq_mask_no_end,batch_size,max_len,mask_id):
    proba = torch.randint(low=0, high=11, size=(idx_target.shape[0],))/10
    proba = proba[:, None].cuda()
    mask = torch.bernoulli(proba * torch.ones(idx_target[...,idx_pred].shape,device=idx_target.device))
    mask = torch.logical_or(mask, ~seq_mask_no_end).int().unsqueeze(-1).repeat(1,1,5)
    mask[idx_pred] = torch.zeros_like(mask[idx_pred])
    input_indices = mask*idx_target+(1-mask)*mask_id
    idx_target = input_indices

In [11]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

# https://github.com/openai/CLIP/issues/111
class TextCLIP(torch.nn.Module):
    def __init__(self, model) :
        super(TextCLIP, self).__init__()
        self.model = model
        
    def forward(self,text):
        with torch.no_grad():
            word_emb = self.model.token_embedding(text).type(self.model.dtype)
            word_emb = word_emb + self.model.positional_embedding.type(self.model.dtype)
            word_emb = word_emb.permute(1, 0, 2)  # NLD -> LND
            word_emb = self.model.transformer(word_emb)
            word_emb = self.model.ln_final(word_emb).permute(1, 0, 2).float()
            enctxt = self.model.encode_text(text).float()
        return enctxt, word_emb
clip_model = TextCLIP(clip_model)


In [18]:
gt_token =[]
pred_token = []
for nb_iter in tqdm(range(1, args.total_iter + 1), position=0, leave=True):
    batch = next(train_loader_iter)
    clip_text, m_tokens, m_tokens_len = batch
    m_tokens, m_tokens_len = m_tokens.cuda(), m_tokens_len.cuda()
    bs = m_tokens.shape[0]
    target = m_tokens    # (bs, 26)
    target = target.cuda()
    idx_pred=torch.randint(0, 5, (1,)).item()
    print('idx_pred',idx_pred)
    target_pred= target[..., idx_pred].clone()
    gt_token.append(target.clone())
    pred_token.append(target.clone())
    mask_id = get_model(net).vqvae.num_code + 2
    batch_size, max_len = target.shape[:2]
    seq_mask_no_end = generate_src_mask(max_len, m_tokens_len)
    seq_mask = generate_src_mask(max_len, m_tokens_len+1)
    mask_token=mask_target(target,idx_pred,seq_mask_no_end,batch_size,max_len,m_tokens_len,mask_id)
    mask_else(target, idx_pred,seq_mask_no_end,batch_size,max_len,mask_id)

    # Random Drop Text
    text_mask = np.random.random(len(clip_text)) > .1
    clip_text = np.array(clip_text)
    clip_text[~text_mask] = ''
    
    text = clip.tokenize(clip_text, truncate=True).cuda()
    feat_clip_text, word_emb = clip_model(text)
    att_txt = None #proba != 1 # CFG: torch.rand((seq_mask.shape[0], 1)) > 0.1
    #[bs,50,8092]
    cls_pred = trans_encoder(target, feat_clip_text, src_mask = seq_mask, att_txt=att_txt, word_emb=word_emb)[:, 1:] #, txt_mark=txt_mark
    cls_pred = cls_pred[:,:,idx_pred,:].squeeze(2)

    # [INFO] Compute xent loss as a batch
    weights = seq_mask_no_end / (seq_mask_no_end.sum(-1).unsqueeze(-1) * seq_mask_no_end.shape[0])
    cls_pred_seq_masked = cls_pred[seq_mask_no_end, :].view(-1, cls_pred.shape[-1])
    target_seq_masked = target_pred[seq_mask_no_end]
    weight_seq_masked = weights[seq_mask_no_end]
    num_classes = cls_pred.shape[-1]

    probs_seq_masked = torch.softmax(cls_pred_seq_masked, dim=-1)
    _, cls_pred_seq_masked_index = torch.max(probs_seq_masked, dim=-1)
    target_seq_masked = torch.masked_select(target_pred, seq_mask_no_end)
    right_seq_masked = (cls_pred_seq_masked_index == target_seq_masked).sum()
    print(cls_pred_seq_masked_index.shape)
    print(pred_token[-1].shape)
    pred_token[-1][seq_mask_no_end, idx_pred] = cls_pred_seq_masked_index
    print('ACC/every_token', right_seq_masked*100/seq_mask_no_end.sum())


  0%|          | 0/10 [00:00<?, ?it/s]

idx_pred 3


IndexError: index 3 is out of bounds for dimension 0 with size 1

In [ ]:
for gt,pred in zip(gt_token,pred_token):
    pred_pose = net(pred_token, type='decode')
    gt_pose = net(gt_token, type='decode')
    _motions = pred_pose_upper[i, :m_length[i]].detach().cpu().numpy() * val_loader.dataset.std + val_loader.dataset.mean
    _motions = recover_from_ric(torch.from_numpy(_motions).float(), 22).numpy()
    animate3d(_motions, BONE_LINK=t2m_bone, first_total_standard=27, axis_visible=True)